# Deep Learning with R

For R users, there hasn’t been a production grade solution for deep
learning (sorry [MXNET](http://mxnet.io/api/r/index.html)). This post
introduces the Keras interface for R and how it can be used to perform
image classification. The post ends by providing some code snippets that
show Keras is intuitive and powerful 💪🏽.

### Tensorflow

Last January, [Tensorflow for R](https://github.com/rstudio/tensorflow)
was released, which provided access to the Tensorflow API from R. This
was signficant, as Tensorflow is the most popular library for deep
learning. However, for most R users, the Tensorflow for R interface was
not very R like. 🤢 Take a look at this code chunk for training a model:

    cross_entropy <- tf$reduce_mean(-tf$reduce_sum(y_ * tf$log(y_conv), reduction_indices=1L))
    train_step <- tf$train$AdamOptimizer(1e-4)$minimize(cross_entropy)
    correct_prediction <- tf$equal(tf$argmax(y_conv, 1L), tf$argmax(y_, 1L))
    accuracy <- tf$reduce_mean(tf$cast(correct_prediction, tf$float32))
    sess$run(tf$global_variables_initializer())

    for (i in 1:20000) {
      batch <- mnist$train$next_batch(50L)
      if (i %% 100 == 0) {
        train_accuracy <- accuracy$eval(feed_dict = dict(
            x = batch[[1]], y_ = batch[[2]], keep_prob = 1.0))
        cat(sprintf("step %d, training accuracy %g\n", i, train_accuracy))
      }
      train_step$run(feed_dict = dict(
        x = batch[[1]], y_ = batch[[2]], keep_prob = 0.5))
    }

    test_accuracy <- accuracy$eval(feed_dict = dict(
         x = mnist$test$images, y_ = mnist$test$labels, keep_prob = 1.0))
    cat(sprintf("test accuracy %g", test_accuracy))

Yikes!

Unless you are familiar with tensorflow, it’s not readily apparent what
is going on. A quick search on Github finds less than a 100 code results
using tensorflow for R. 😔

### Keras

All this is going to change with Keras and R! ☺️

For background, [Keras](https://keras.io/index.html) is a high-level
neural network API that is designed for experimentation and can run on
top of Tensorflow. Keras is what data scientists like to use. 🤓 Keras
has grown in popularity and supported on a wide set of platforms
including Tensorflow, CNTK, Apple’s CoreML, and Theano. It is becoming
the de factor language for deep learning.

As a simple example, here is the code to train a model in Keras:

    model_top %>% fit(
            x = train_x, y = train_y,
            epochs=epochs, 
            batch_size=batch_size,
            validation_data=valid)

### Image Classification with Keras

So if you are still with me, let me show you how to build deep learning
models using R, Keras, and Tensorflow together. You will find a Github
repo at <https://github.com/rajshah4/image_keras/> that contains the
code and data you will need. Included is an [R
notebook](http://htmlpreview.github.io/?https://github.com/rajshah4/image_keras/blob/master/Rnotebook.nb.html)
(and Python notebooks) that walks through building an image classifier
(telling 🐱 from 🐶), but can easily be generalized to other images. The
walk through includes advanced methods that are commonly used for
production deep learning work including:

-   augmenting data
-   using the bottleneck features of a pre-trained network
-   fine-tuning the top layers of a pre-trained network
-   saving weights of models

### Code Snippets of Keras

The R interface to Keras truly makes it easy to build deep learning
models in R. Here are some code snippets based on my example of building
an image classifier to illustrate how intuitive and useful Keras for R
is:

To load 🖼 from a folder:

    train_generator <- flow_images_from_directory(train_directory, generator = image_data_generator(), target_size = c(img_width, img_height), color_mode = "rgb",
      class_mode = "binary", batch_size = batch_size, shuffle = TRUE,
      seed = 123)

To define a simple convolutional neural network:

    model <- keras_model_sequential()

    model %>%
      layer_conv_2d(filter = 32, kernel_size = c(3,3), input_shape = c(img_width, img_height, 3)) %>%
      layer_activation("relu") %>%
      layer_max_pooling_2d(pool_size = c(2,2)) %>% 
      
      layer_conv_2d(filter = 32, kernel_size = c(3,3)) %>%
      layer_activation("relu") %>%
      layer_max_pooling_2d(pool_size = c(2,2)) %>%
      
      layer_conv_2d(filter = 64, kernel_size = c(3,3)) %>%
      layer_activation("relu") %>%
      layer_max_pooling_2d(pool_size = c(2,2)) %>%
      
      layer_flatten() %>%
      layer_dense(64) %>%
      layer_activation("relu") %>%
      layer_dropout(0.5) %>%
      layer_dense(1) %>%
      layer_activation("sigmoid")

To augment data:

    augment <- image_data_generator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=TRUE)

To load a pretrained network:

    model_vgg <- application_vgg16(include_top = FALSE, weights = "imagenet")

To save model weights:

    save_model_weights_hdf5(model_ft, 'finetuning_30epochs_vggR.h5', overwrite = TRUE)

The Keras for R interface makes it much easier for R users to build and
refine deep learning models. Its no longer necessary to force everyone
to use Python to build, refine, and test deep learning models. I really
think this will open up deep learning to a wider audience that was a bit
apprehensive on using python.

To start with, you can grab my
[repo](https://github.com/rajshah4/image_keras/), fire up RStudio (or
your IDE of choice), and go build a simple classifier using Keras. There
are also a wealth of [other
examples](https://rstudio.github.io/keras/articles/examples/index.html)
such as [generating
text](https://rstudio.github.io/keras/articles/examples/lstm_text_generation.html)
from Nietzsche’s writings, [deep
dreaming](https://rstudio.github.io/keras/articles/examples/deep_dream.html),
or creating a [variational
encoder](https://rstudio.github.io/keras/articles/examples/variational_autoencoder.html).

So for now, give it a spin!

An earlier version of this post was posted at
[Datascience+](https://datascienceplus.com/deep-learning-with-r/).